In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import os
import gmaps
#import gmplot
#from geolocation.main import GoogleMaps
#from geolocation.distance_matrix.client import DistanceMatrixApiClient
from yelpapi import YelpAPI
from pprint import pprint
from config import apikey
from config import gkey
import csv
from yelp.client import Client

In [2]:
# Get yo' Key
yelp_api = YelpAPI(apikey)
#print(yelp_api)

In [3]:
# Print each city's input into different lists. 
# We also need information on the restaurant's name, rating, location, lat/long and category
# To do this we're going to set up a list of cities, and a dictionary to hold it all in.
yelp_api = YelpAPI(apikey)
city_names = ["Oceanside", "Encinitas", "Carlsbad"]
data = {
    city: {
        'names': [],
        'ratings': [],
        'categories': [],
        'location':[],
        'lat':[],
        'lng':[]
    }
    for city in city_names
}
# Then we're gonna print just to see that our lists and dictionaries are set up correctly
print(json.dumps(data, indent=4))

#Now we just run a loop through the city names to append the data into the city dictionaries where they belong
for city in city_names:
    search_results = yelp_api.search_query(location=city, 
                                           term="restaurant", 
                                           limit = 50) 
                                           #offset=100)
    for b in search_results['businesses']:
        try:
            data[city]['names'].append(b["name"])
            data[city]['ratings'].append(b["rating"])
            data[city]['categories'].append(b["categories"][0]['title']+"/"+b["categories"][0]["alias"])
            data[city]['location'].append(b['location']['city'])
            data[city]['lat'].append(b['coordinates']['latitude'])
            data[city]['lng'].append(b['coordinates']['longitude'])
        except:
            data[city]['names'].append(np.nan)
            data[city]['ratings'].append(np.nan)
            data[city]['categories'].append(np.nan)
            data[city]['location'].append(np.nan)
            data[city]['lat'].append(np.nan)
            data[city]['lng'].append(np.nan)

# You can run a test here for length checks.

# print(len(search_results['businesses']))
# print(len(name))
# print(len(rating))
# print(len(category))
# print(len(city_list))

# Then we pretty print the dictionaries to make sure they're all populated
pprint(data, indent=4)

{
    "Oceanside": {
        "names": [],
        "ratings": [],
        "categories": [],
        "location": [],
        "lat": [],
        "lng": []
    },
    "Encinitas": {
        "names": [],
        "ratings": [],
        "categories": [],
        "location": [],
        "lat": [],
        "lng": []
    },
    "Carlsbad": {
        "names": [],
        "ratings": [],
        "categories": [],
        "location": [],
        "lat": [],
        "lng": []
    }
}
{   'Carlsbad': {   'categories': [   'American (New)/newamerican',
                                      'Seafood/seafood',
                                      'Gastropubs/gastropubs',
                                      'Comfort Food/comfortfood',
                                      'Cocktail Bars/cocktailbars',
                                      'American (New)/newamerican',
                                      'Pubs/pubs',
                                      'Steakhouses/steak',
                           

In [4]:
# Now that we have everything in seperate dictionaries, let's put it together into 3 seperate dataframes
Oceanside_df = pd.DataFrame.from_dict(data["Oceanside"])
Carlsbad_df = pd.DataFrame.from_dict(data["Carlsbad"])
Encinitas_df = pd.DataFrame.from_dict(data["Encinitas"])

# We'll print the numbers to make sure they're at 50 each
print(len(Oceanside_df))
print(len(Carlsbad_df))
print(len(Encinitas_df))

# Then we're dropping any null values
Oceanside_df = Oceanside_df.dropna(how='any')
Carlsbad_df = Carlsbad_df.dropna(how='any')
Encinitas_df = Encinitas_df.dropna(how='any')

# And printing again to make sure those are dropped if there are any
print(len(Oceanside_df))
print(len(Carlsbad_df))
print(len(Encinitas_df))

50
50
50
50
50
50


In [5]:
# Since some of our data isn't from the cities we're interested in, let's fix that.

# First we'll set the index to "location"
Oceanside_df = Oceanside_df.set_index("location")
Carlsbad_df = Carlsbad_df.set_index("location")
Encinitas_df= Encinitas_df.set_index("location")

# Then we're going to run a loc on the cities we want
# This will make sure we only have the cities we are interested in
Oceanside_df=Oceanside_df.loc["Oceanside",:]
Carlsbad_df=Carlsbad_df.loc["Carlsbad",:]
Encinitas_df=Encinitas_df.loc["Encinitas",:]

# We'll print the lengths to see what's left, and the head to make sure the DFs are what we want

print(len(Oceanside_df))
print(len(Carlsbad_df))
print(len(Encinitas_df))

Oceanside_df.head(20)

46
47
44


,names,ratings,categories,lat,lng
location,,,,,
Oceanside,Flying Pig Pub & Kitchen,4.5,Pubs/pubs,33.188440,-117.374070
Oceanside,Miss Kim's by That Boy Good,4.5,Barbeque/bbq,33.196043,-117.380008
Oceanside,Local Tap House & Kitchen,4.5,American (New)/newamerican,33.192741,-117.376506
Oceanside,Rockin' Baja Lobster,4.0,Mexican/mexican,33.204210,-117.390270
Oceanside,Miramar Fish Tacos & Beer,4.5,Seafood/seafood,33.226692,-117.326496
Oceanside,Felix's BBQ With Soul,4.5,Southern/southern,33.210161,-117.311183
Oceanside,Fratelli's Italian Kitchen,4.5,Italian/italian,33.228141,-117.324574
Oceanside,The Miller's Table,5.0,Wine Bars/wine_bars,33.190360,-117.374490
Oceanside,Panca Peruvian Cuisine & Rotisserie,4.5,Peruvian/peruvian,33.173822,-117.360565


In [6]:
# Let's see who our top 5 competitors are in each city
# We'll start with Oceanside
SortedOceanside=Oceanside_df.sort_values("ratings", ascending=False)
SortedCarlsbad=Carlsbad_df.sort_values("ratings",ascending=False)
SortedEncinitas=Encinitas_df.sort_values("ratings",ascending=False)

Top5Oceanside = SortedOceanside.head(5).reset_index()
Top5Oceanside

,location,names,ratings,categories,lat,lng
0,Oceanside,Rosaria Pizza,5.0,Pizza/pizza,33.195660,-117.380850
1,Oceanside,Mangia e Bevi,5.0,Italian/italian,33.210110,-117.311620
2,Oceanside,The Miller's Table,5.0,Wine Bars/wine_bars,33.190360,-117.374490
3,Oceanside,O'side Sports Bar & Grill,5.0,Sports Bars/sportsbars,33.194526,-117.378585
4,Oceanside,Board & Brew,5.0,Beer Bar/beerbar,33.186110,-117.328900


In [7]:
Top5Carlsbad = SortedCarlsbad.head(5).reset_index()
Top5Carlsbad

,location,names,ratings,categories,lat,lng
0,Carlsbad,Harumama Carlsbad,5.0,Ramen/ramen,33.161560,-117.346860
1,Carlsbad,Bigfoot Natural Cafe,5.0,Vegan/vegan,33.075410,-117.237240
2,Carlsbad,2051 Cafe,5.0,Cafes/cafes,33.122497,-117.276417
3,Carlsbad,Full Metal Burgers,5.0,Food Trucks/foodtrucks,33.212524,-117.291587
4,Carlsbad,Campfire,4.5,American (New)/newamerican,33.161990,-117.351050


In [8]:
Top5Encinitas = SortedEncinitas.head(5).reset_index()
Top5Encinitas

,location,names,ratings,categories,lat,lng
0,Encinitas,Chiosco Caffè,5.0,Cafes/cafes,33.053910,-117.261000
1,Encinitas,Buona Forchetta,5.0,Pizza/pizza,33.053590,-117.296590
2,Encinitas,Plum Encinitas,5.0,Thai/thai,33.045510,-117.293450
3,Encinitas,Hammer's NY Pizza,4.5,Pizza/pizza,33.049890,-117.259680
4,Encinitas,CAVA,4.5,Mediterranean/mediterranean,33.065783,-117.265282


In [14]:
Oceanside_df = Oceanside_df.reset_index()
GroupedOceanside = Oceanside_df.groupby("categories").count()
GroupedOceanside
#GroupedOceanside.head()

,level_0,index,location,names,ratings,lat,lng
categories,,,,,,,
American (New)/newamerican,5,5,5,5,5,5,5
American (Traditional)/tradamerican,2,2,2,2,2,2,2
Barbeque/bbq,2,2,2,2,2,2,2
Beer Bar/beerbar,1,1,1,1,1,1,1
Breakfast & Brunch/breakfast_brunch,2,2,2,2,2,2,2
Breweries/breweries,1,1,1,1,1,1,1
Burgers/burgers,1,1,1,1,1,1,1
Caribbean/caribbean,1,1,1,1,1,1,1
Cheesesteaks/cheesesteaks,1,1,1,1,1,1,1


In [15]:
for data in Oceanside_df:
    hashStr = hash(Oceanside_df["categories"])
    r = (hashStr & 0xFF0000) >> 16;
    g = (hashStr & 0x00FF00) >> 8;
    b = hashStr & 0x0000FF;

#print(r,g,b)

24 118 25


In [10]:
# Now we do heatmaps. First we're loading up the figures we want
gmaps.configure(api_key=gkey)
Carlsbad_coordinates = (33.1580933,-117.339839)
Carlsbad_fig=gmaps.figure(center=Carlsbad_coordinates, zoom_level=14)
Carlsbad_fig

Figure(layout=FigureLayout(height='420px'))

In [73]:
Oceanside_coordinates = (33.1959, -117.3795)
Oceanside_fig = gmaps.figure(center=Oceanside_coordinates, zoom_level=12)
Oceanside_fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
Encinitas_coordinates = (33.0370, -117.2920)
Encinitas_fig = gmaps.figure(center=Encinitas_coordinates, zoom_level=14)
Encinitas_fig

Figure(layout=FigureLayout(height='420px'))

In [74]:
Oceanside_locations = Oceanside_df[["lat","lng"]].astype(float)
# create empty lists for colors and category
# store the category and the color in their respective lists
# turn lists into a legend


for i, row in Oceanside_df.iterrows():
    print(row['categories'])
    Olat = row['lat']
    Olng = row['lng']
    hashStr = hash(row['categories'])
    #Friend helped me with the bitshift shenanigans
    # Hash turns a string into an int, it puts numbers based on what the letters are
    # We're then going to manipulate the hashStr to get our RGB values
    # & is a bit mask, whatever numbers match up with the FFs are kept, but in binary code. 
    # >> is a bitshift, where we're taking the binary, and we're moving them 16 zeroes over to the left 
    r = (hashStr & 0xFF0000) >> 16;
    g = (hashStr & 0x00FF00) >> 8;
    b = hashStr & 0x0000FF;
    Oceanside_fig.add_layer(gmaps.symbol_layer([(Olat, Olng)], fill_color=(r,b,g), stroke_color = 'black', scale =5))
    #Oceanside_fig.add_layer(gmaps.symbol_layer(Oceanside_locations, fill_color="({}, {}, {})".format(r, g,b), stroke_color = 'rgba(0,0,0,0)', scale =.05))
Oceanside_fig
    

Pubs/pubs
Barbeque/bbq
American (New)/newamerican
Mexican/mexican
Seafood/seafood
Southern/southern
Italian/italian
Wine Bars/wine_bars
Peruvian/peruvian
American (New)/newamerican
Steakhouses/steak
Japanese/japanese
American (New)/newamerican
Ramen/ramen
Ramen/ramen
Gastropubs/gastropubs
American (New)/newamerican
Italian/italian
American (Traditional)/tradamerican
Breweries/breweries
Caribbean/caribbean
American (New)/newamerican
Burgers/burgers
Seafood/seafood
Thai/thai
Barbeque/bbq
Indonesian/indonesian
Breakfast & Brunch/breakfast_brunch
Fish & Chips/fishnchips
Seafood/seafood
Sushi Bars/sushi
Cocktail Bars/cocktailbars
Mexican/mexican
Mexican/mexican
Breakfast & Brunch/breakfast_brunch
Hawaiian/hawaiian
Vietnamese/vietnamese
Pizza/pizza
Beer Bar/beerbar
American (Traditional)/tradamerican
Thai/thai
Sports Bars/sportsbars
Cheesesteaks/cheesesteaks
Pizza/pizza
Vietnamese/vietnamese
Ramen/ramen


Figure(layout=FigureLayout(height='420px'))

In [27]:
#Oceanside_df = Oceanside_df.set_index("categories")
print(Oceanside_df["categories"])

0                               Pubs/pubs
1                            Barbeque/bbq
2              American (New)/newamerican
3                         Mexican/mexican
4                         Seafood/seafood
5                       Southern/southern
6                         Italian/italian
7                     Wine Bars/wine_bars
8                       Peruvian/peruvian
9              American (New)/newamerican
10                      Steakhouses/steak
11                      Japanese/japanese
12             American (New)/newamerican
13                            Ramen/ramen
14                            Ramen/ramen
15                  Gastropubs/gastropubs
16             American (New)/newamerican
17                        Italian/italian
18    American (Traditional)/tradamerican
19                    Breweries/breweries
20                    Caribbean/caribbean
21             American (New)/newamerican
22                        Burgers/burgers
23                        Seafood/

In [33]:
# Let's break the dataframes into their categories
# We're going to have to get a list of each 

Barbeque/bbq


0                               Pubs/pubs
1                            Barbeque/bbq
2              American (New)/newamerican
3                         Mexican/mexican
4                         Seafood/seafood
5                       Southern/southern
6                         Italian/italian
7                     Wine Bars/wine_bars
8                       Peruvian/peruvian
9              American (New)/newamerican
10                      Steakhouses/steak
11                      Japanese/japanese
12             American (New)/newamerican
13                            Ramen/ramen
14                            Ramen/ramen
15                  Gastropubs/gastropubs
16             American (New)/newamerican
17                        Italian/italian
18    American (Traditional)/tradamerican
19                    Breweries/breweries
20                    Caribbean/caribbean
21             American (New)/newamerican
22                        Burgers/burgers
23                        Seafood/

In [75]:
hashStr = hash("hello world")
print(hashStr & 0xFF0000)
print(hashStr & 0x00FF00)
print(hashStr & 0x0000FF)
print('-------')

print((hashStr & 0xFF0000) >> 16)
print((hashStr & 0x00FF00) >> 8)
print(hashStr & 0x0000FF)

1572864
30208
25
-------
24
118
25
